# simplesentiment

## import

In [66]:
import pandas as pd
import numpy as np
import re
import sys
import warnings
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
import nltk
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import word_tokenize
from itertools import product

___
## data reading

In [12]:
df_train = pd.read_csv('products_sentiment_train.tsv', '\t', names=['text', 'label'])
df_test = pd.read_csv('products_sentiment_test.tsv', '\t')

___
## baseline

Для начала попробуем 2 простых алгоритм с CountVectorizer/TfidfVectorizer и LogisticRegression (парметры по умолчанию), с который будем сравнивать другие модели

In [13]:
cvec = CountVectorizer()
cvec_features = cvec.fit_transform(df_train.text)
print('CountVectorizer feature number:', cvec_features.shape[1])

CountVectorizer feature number: 3973


accuracy с CountVectorizer

In [14]:
baseline_acc_cvec = cross_val_score(LogisticRegression(), cvec_features, df_train.label, scoring='accuracy').mean()
print('Baseline accuracy 1: %.3f' % baseline_acc_cvec)

Baseline accuracy 1: 0.768


accuracy с TfidfVectorizer

In [15]:
tfidf_baseline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', LogisticRegression())
])
baseline_acc_tfidf = cross_val_score(tfidf_baseline, df_train.text, df_train.label, scoring='accuracy').mean()
print('Baseline accuracy 2: %.3f' % baseline_acc_tfidf)

Baseline accuracy 2: 0.766


Результат примерно одинаковый. Продолжим перебирать параметры

___
## settings

Попробуем различные варианты извлечения признаков, в том числе стемминг и лемматизацию. Для этого потребуются вспомогательные функции

In [22]:
porter = PorterStemmer()
lancaster = LancasterStemmer()
wn_lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def wordnet_lemmatize(text):
    global wn_lemmatizer
    tokens = word_tokenize(text)
    nltk_tags = nltk.pos_tag(tokens)
    return ' '.join([wn_lemmatizer.lemmatize(t[0], get_wordnet_pos(t[1])) for t in nltk_tags])

def porter_stem(text):
    global porter
    tokens = word_tokenize(text)
    return ' '.join([porter.stem(t) for t in tokens])

def lancaster_stem(text):
    global lancaster
    tokens = word_tokenize(text)
    return ' '.join([lancaster.stem(t) for t in tokens])

В качестве стемминга попробуем PorterStemmer и LancasterStemmer. Подготовим предобработанные датасэты для стемминга и лемматизации

In [23]:
train_lemmatized = df_train.copy(deep=True)
train_lemmatized.text = train_lemmatized.text.apply(wordnet_lemmatize)

In [24]:
train_porter = df_train.copy(deep=True)
train_porter.text = train_porter.text.apply(porter_stem)

In [25]:
train_lancaster = df_train.copy(deep=True)
train_lancaster.text = train_lancaster.text.apply(lancaster_stem)

___
## stemming and lemmatization

Для начала попробуем оценить как влияет лемматизация и стемминг на качество модели. Для этого следующие параметры:
* перпроцессинг: без препроцессинга, лемматизация, стемминг Портера, стемминг ланкастера
* очистка данных: да, нет
* векотризация: CountVectorizer, TfidfVectorizer
* классификатор: LogisticRegression, SGDClassifier, LinearSVC, MultinomialNB

In [29]:
random_state = 42
data_frames_map = {
    'original': df_train,
    'lemmatization': train_lemmatized,
    'porter': train_porter,
    'lancaster': train_lancaster
}
data_frames = list(data_frames_map.keys())
clear_data = (False, True)
vectorizers = (CountVectorizer, TfidfVectorizer)
classifiers = (
    LogisticRegression(random_state=random_state),
    SGDClassifier(random_state=random_state),
    LinearSVC(random_state=random_state),
    MultinomialNB()
)

raw_params = list(zip(
    *list(product(data_frames, clear_data, vectorizers, classifiers))
))

preprocessing_params = pd.DataFrame({
    'df': raw_params[0],
    'clear_data': raw_params[1],
    'vectorizer': raw_params[2],
    'classifier': raw_params[3]
})

preprocessing_params.sample(frac=0.1)

df  clear_data                                         vectorizer  \
52  lancaster       False  <class 'sklearn.feature_extraction.text.TfidfV...   
48  lancaster       False  <class 'sklearn.feature_extraction.text.CountV...   
11   original        True  <class 'sklearn.feature_extraction.text.CountV...   
12   original        True  <class 'sklearn.feature_extraction.text.TfidfV...   
47     porter        True  <class 'sklearn.feature_extraction.text.TfidfV...   
60  lancaster        True  <class 'sklearn.feature_extraction.text.TfidfV...   

                             classifier  
52  LogisticRegression(random_state=42)  
48  LogisticRegression(random_state=42)  
11                      MultinomialNB()  
12  LogisticRegression(random_state=42)  
47                      MultinomialNB()  
60  LogisticRegression(random_state=42)

Добавим функции для оценки качества модели

In [45]:
total = preprocessing_params.shape[0]
counter = 1

def remove_punctuation(df: pd.DataFrame):
    def transform_text(text):
        text = re.sub('[^a-zA-Z]', ' ', text)
        return re.sub(r'\s+', ' ', text)
    df.text = df.text.apply(transform_text)
    return df

def preprocessing_accuracy(row):
    global counter
    sys.stdout.write('progress: %.3f %%\r' % (counter/total * 100))
    counter += 1
    df = data_frames_map[row.df]
    if row.clear_data:
        df = remove_punctuation(df)
        stop_words = 'english'
    else:
        stop_words = None
    vectorizer = row.vectorizer(stop_words=stop_words)
    estimator = Pipeline([
        ('vectorizer', vectorizer),
        ('classifier', row.classifier)
    ])
    return cross_val_score(estimator, df.text, df.label, scoring='accuracy').mean()

In [46]:
%%time
warnings.filterwarnings('ignore')
preprocessing_params['score'] = preprocessing_params.apply(preprocessing_accuracy, axis=1)
print()
warnings.filterwarnings('default')

progress: 100.000 %
Wall time: 17.2 s


In [47]:
top_models = preprocessing_params.sort_values('score', ascending=False)[:10]
top_models

df  clear_data  \
35         porter       False   
19  lemmatization       False   
51      lancaster       False   
3        original       False   
38         porter       False   
22  lemmatization       False   
54      lancaster       False   
32         porter       False   
48      lancaster       False   
16  lemmatization       False   

                                           vectorizer  \
35  <class 'sklearn.feature_extraction.text.CountV...   
19  <class 'sklearn.feature_extraction.text.CountV...   
51  <class 'sklearn.feature_extraction.text.CountV...   
3   <class 'sklearn.feature_extraction.text.CountV...   
38  <class 'sklearn.feature_extraction.text.TfidfV...   
22  <class 'sklearn.feature_extraction.text.TfidfV...   
54  <class 'sklearn.feature_extraction.text.TfidfV...   
32  <class 'sklearn.feature_extraction.text.CountV...   
48  <class 'sklearn.feature_extraction.text.CountV...   
16  <class 'sklearn.feature_extraction.text.CountV...   

                             classifier   score  
35                      MultinomialNB()  0.7820  
19                      MultinomialNB()  0.7800  
51                      MultinomialNB()  0.7785  
3                       MultinomialNB()  0.7755  
38           LinearSVC(random_state=42)  0.7745  
22           LinearSVC(random_state=42)  0.7725  
54           LinearSVC(random_state=42)  0.7725  
32  LogisticRegression(random_state=42)  0.7710  
48  LogisticRegression(random_state=42)  0.7700  
16  LogisticRegression(random_state=42)  0.7695

___
## ngramms

Стемминг/лемматизация явно улучшают качество модели. Также интересно, что в топе лучше всего себя показывает связка наивного байеса с мешком слов, а также LinearSVC с tfidf. При этом чистка данных не помогает.

Попробуем теперь перебрать различные варианты n-грамм. И проверять их будем на параметрах которые уже попали в топ:
* перпроцессинг: лемматизация, стемминг Портера, стемминг Ланкастера
* модель : LinearSVC + tfidf, MultinomialNB + CountVectorizer

In [48]:
random_state = 42

def TfidfLinearSVC(**kwargs):
    return Pipeline([
        ('vectorizer', TfidfVectorizer(**kwargs)),
        ('classifier', LinearSVC(random_state=random_state))
    ])

def CounterMultinomialNB(**kwargs):
    return Pipeline([
        ('vectorizer', CountVectorizer(**kwargs)),
        ('classifier', MultinomialNB())
    ])

In [50]:
data_frames_map = {
    'lemmatization': train_lemmatized,
    'porter': train_porter,
    'lancaster': train_lancaster
}
data_frames = list(data_frames_map.keys())
wgrams = ((1, 1), (1, 2), (1, 3))
cgrams = ((2, 3), (2, 5), (3, 5))
models = (TfidfLinearSVC, CounterMultinomialNB)
raw_params = list(zip(
    *list(product(data_frames, wgrams, [None], models)) + \
     list(product(data_frames, [None], cgrams, models))
))

ngramm_params = pd.DataFrame({
    'df': raw_params[0],
    'wgrams': raw_params[1],
    'cgrams': raw_params[2],
    'model': raw_params[3]
})
ngramm_params.sample(n=5)

df  wgrams  cgrams  \
12      lancaster  (1, 1)    None   
14      lancaster  (1, 2)    None   
26         porter    None  (2, 5)   
17      lancaster  (1, 3)    None   
0   lemmatization  (1, 1)    None   

                                                model  
12    <function TfidfLinearSVC at 0x00000245B4403940>  
14    <function TfidfLinearSVC at 0x00000245B4403940>  
26    <function TfidfLinearSVC at 0x00000245B4403940>  
17  <function CounterMultinomialNB at 0x00000245B4...  
0     <function TfidfLinearSVC at 0x00000245B4403940>

In [51]:
total = ngramm_params.shape[0]
counter = 1

def ngramm_accuracy(row):
    global counter
    global total
    sys.stdout.write('progress: %.3f %%\r' % (counter/total * 100))
    counter += 1

    df = data_frames_map[row.df]
    ngramm, analyzer = (row.wgrams, 'word') if row.wgrams is not None else (row.cgrams, 'char_wb')
    model = row.model(ngram_range=ngramm, analyzer=analyzer)
    return cross_val_score(model, df.text, df.label, scoring='accuracy').mean()

In [52]:
%%time
warnings.filterwarnings('ignore')
ngramm_params['score'] = ngramm_params.apply(ngramm_accuracy, axis=1)
print()
warnings.filterwarnings('default')

progress: 100.000 %
Wall time: 37.4 s


In [53]:
ngramm_params.sort_values('score', ascending=False)[:10]

df  wgrams  cgrams  \
2   lemmatization  (1, 2)    None   
4   lemmatization  (1, 3)    None   
14      lancaster  (1, 2)    None   
16      lancaster  (1, 3)    None   
10         porter  (1, 3)    None   
8          porter  (1, 2)    None   
23  lemmatization    None  (3, 5)   
28         porter    None  (3, 5)   
21  lemmatization    None  (2, 5)   
33      lancaster    None  (2, 5)   

                                                model   score  
2     <function TfidfLinearSVC at 0x00000245B4403940>  0.7915  
4     <function TfidfLinearSVC at 0x00000245B4403940>  0.7895  
14    <function TfidfLinearSVC at 0x00000245B4403940>  0.7885  
16    <function TfidfLinearSVC at 0x00000245B4403940>  0.7870  
10    <function TfidfLinearSVC at 0x00000245B4403940>  0.7865  
8     <function TfidfLinearSVC at 0x00000245B4403940>  0.7845  
23  <function CounterMultinomialNB at 0x00000245B4...  0.7845  
28    <function TfidfLinearSVC at 0x00000245B4403940>  0.7835  
21  <function CounterMultinomialNB at 0x00000245B4...  0.7830  
33  <function CounterMultinomialNB at 0x00000245B4...  0.7830

In [54]:
feature_matrix = TfidfVectorizer(ngram_range=(1,2)).fit_transform(train_lemmatized.text)
print('current best result: %.4f' % ngramm_params.score.sort_values(ascending=False).values[0])
print('current number of features:', feature_matrix.shape[1])

current best result: 0.7915
current number of features: 20299


___
## dimension reduction and trees

На данный момент победитель с небольшим отрывом: TfidfVectorizer + LinearSVC с лемматизацией и 1-2 словограммами. Попробуем уменьшить количество признаков и обучить на полученных данных ансамбли деревьев

In [55]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

ngrams = ((1, 1), (1, 2), (1, 3))
max_features = (100, 200, 500)
vectorizers = (CountVectorizer, TfidfVectorizer)
classifiers = (RandomForestClassifier(random_state=random_state), GradientBoostingClassifier(random_state=random_state))

raw_params = list(zip(
    *list(product(ngrams, max_features, vectorizers, classifiers))
))

tree_params = pd.DataFrame({
    'ngram': raw_params[0],
    'max_features': raw_params[1],
    'vectorizer': raw_params[2],
    'classifier': raw_params[3]
})

In [56]:
total = tree_params.shape[0]
counter = 1

def tree_accuracy(row):
    global counter
    global total
    sys.stdout.write('progress: %.3f %%\r' % (counter/total * 100))
    counter += 1
    
    model = Pipeline([
        ('vectorizer', row.vectorizer(ngram_range=row.ngram, max_features=row.max_features)),
        ('classifier', row.classifier)
    ])
    return cross_val_score(model, train_lemmatized.text, train_lemmatized.label, scoring='accuracy').mean()

In [57]:
%%time
warnings.filterwarnings('ignore')
tree_params['score'] = tree_params.apply(tree_accuracy, axis=1)
print()
warnings.filterwarnings('default')

progress: 100.000 %
Wall time: 1min 43s


In [58]:
tree_params.sort_values('score', ascending=False)[:10]

ngram  max_features                                         vectorizer  \
8   (1, 1)           500  <class 'sklearn.feature_extraction.text.CountV...   
10  (1, 1)           500  <class 'sklearn.feature_extraction.text.TfidfV...   
32  (1, 3)           500  <class 'sklearn.feature_extraction.text.CountV...   
20  (1, 2)           500  <class 'sklearn.feature_extraction.text.CountV...   
9   (1, 1)           500  <class 'sklearn.feature_extraction.text.CountV...   
34  (1, 3)           500  <class 'sklearn.feature_extraction.text.TfidfV...   
16  (1, 2)           200  <class 'sklearn.feature_extraction.text.CountV...   
22  (1, 2)           500  <class 'sklearn.feature_extraction.text.TfidfV...   
28  (1, 3)           200  <class 'sklearn.feature_extraction.text.CountV...   
33  (1, 3)           500  <class 'sklearn.feature_extraction.text.CountV...   

                                     classifier   score  
8       RandomForestClassifier(random_state=42)  0.7575  
10      RandomForestClassifier(random_state=42)  0.7530  
32      RandomForestClassifier(random_state=42)  0.7460  
20      RandomForestClassifier(random_state=42)  0.7450  
9   GradientBoostingClassifier(random_state=42)  0.7415  
34      RandomForestClassifier(random_state=42)  0.7415  
16      RandomForestClassifier(random_state=42)  0.7390  
22      RandomForestClassifier(random_state=42)  0.7390  
28      RandomForestClassifier(random_state=42)  0.7390  
33  GradientBoostingClassifier(random_state=42)  0.7380

Не лучшие результаты, но можно заметить, что наилучшие результаты показывают ансамбли с максимальным количеством признаков. Можно поробовать увеличить количество признаков, а также попробовать изменить количество деревьев.

In [59]:
for mf in (600, 800):
    for ne in (200, 500):
        model = Pipeline([
            ('vectorizer', CountVectorizer(max_features=mf)),
            ('classifier', RandomForestClassifier(random_state=random_state, n_estimators=ne))
        ])
        print(cross_val_score(model, train_lemmatized.text, train_lemmatized.label).mean())

0.7675
0.7655000000000001
0.7615000000000001
0.7649999999999999


___
## SVD

Качество конечно немного улучшилось, но значимых результатов это не даёт. Попробуем уменьшить размерность с помощью SVD

In [60]:
from sklearn.decomposition import TruncatedSVD

ngrams = ((1, 1), (1, 2), (1, 3))
ncomp = (100, 500, 1000)
classifiers = (RandomForestClassifier(random_state=random_state), LinearSVC(random_state=random_state))
raw_params = list(zip(
    *list(product(ngrams, ncomp, classifiers))
))

pca_params = pd.DataFrame({
    'ngram': raw_params[0],
    'ncomp': raw_params[1],
    'classifier': raw_params[2]
})

In [61]:
total = pca_params.shape[0]
counter = 1

def pca_accuracy(row):
    global counter
    global total
    sys.stdout.write('progress: %.3f %%\r' % (counter/total * 100))
    counter += 1
    
    model = Pipeline([
        ('vectorizer', CountVectorizer(ngram_range=row.ngram)),
        ('transformer', TruncatedSVD(row.ncomp)),
        ('classifier', row.classifier)
    ])
    return cross_val_score(model, train_lemmatized.text, train_lemmatized.label, scoring='accuracy').mean()

In [62]:
%%time
warnings.filterwarnings('ignore')
pca_params['score'] = pca_params.apply(pca_accuracy, axis=1)
print()
warnings.filterwarnings('default')

progress: 100.000 %
Wall time: 10min 44s


In [64]:
pca_params.sort_values('score', ascending=False)[:10]

ngram  ncomp                               classifier   score
15  (1, 3)    500               LinearSVC(random_state=42)  0.7735
1   (1, 1)    100               LinearSVC(random_state=42)  0.7615
17  (1, 3)   1000               LinearSVC(random_state=42)  0.7580
9   (1, 2)    500               LinearSVC(random_state=42)  0.7550
11  (1, 2)   1000               LinearSVC(random_state=42)  0.7540
7   (1, 2)    100               LinearSVC(random_state=42)  0.7535
13  (1, 3)    100               LinearSVC(random_state=42)  0.7485
5   (1, 1)   1000               LinearSVC(random_state=42)  0.7440
3   (1, 1)    500               LinearSVC(random_state=42)  0.7420
0   (1, 1)    100  RandomForestClassifier(random_state=42)  0.7205

___
## FeatureUnion

По прежнему не удалось побить результаты лучшей модели. Последнее что можно попробовать - обединение признаков векторайзера и признаков, которые получаются после SVD

In [67]:
from sklearn.pipeline import FeatureUnion
random_state = 42

def build_feature_matrix(row):
    feature_extractor = Pipeline([
        ('vectorizer', CountVectorizer(ngram_range=row.ngram)),
        ('transformer', FeatureUnion([
            ('tfidf', TfidfTransformer()),
            ('svd', TruncatedSVD(row.ncomp, random_state=random_state))
        ]))
    ])
    return feature_extractor.fit_transform(train_lemmatized.text)

ngrams = ((1, 1), (1, 2), (1, 3))
ncomp = (1, 2, 10, 50, 100, 200, 500)
raw_params = list(zip(
    *list(product(ngrams, ncomp))
))

uf_params = pd.DataFrame({
    'ngram': raw_params[0],
    'ncomp': raw_params[1]
})

uf_params['feature_matrix'] = uf_params.apply(build_feature_matrix, axis=1)

In [68]:
total = uf_params.shape[0]
counter = 1

def uf_accuracy(row):
    global counter
    global total
    sys.stdout.write('progress: %.3f %%\r' % (counter/total * 100))
    counter += 1
    return cross_val_score(LinearSVC(random_state=random_state, max_iter=100000, tol=1e-7), row.feature_matrix, train_lemmatized.label, scoring='accuracy').mean()

In [69]:
%%time
warnings.filterwarnings('ignore')
uf_params['score'] = uf_params.apply(uf_accuracy, axis=1)
print()
warnings.filterwarnings('default')

progress: 100.000 %
Wall time: 1min 4s


In [70]:
uf_params.sort_values('score', ascending=False)[:10]

ngram  ncomp                                     feature_matrix   score
11  (1, 2)    100    (0, 1394)\t0.39145456656817385\n  (0, 1403)\...  0.7895
18  (1, 3)    100    (0, 3198)\t0.33933345622928995\n  (0, 3217)\...  0.7890
10  (1, 2)     50    (0, 1394)\t0.39145456656817385\n  (0, 1403)\...  0.7880
9   (1, 2)     10    (0, 1394)\t0.39145456656817385\n  (0, 1403)\...  0.7835
8   (1, 2)      2    (0, 1394)\t0.39145456656817385\n  (0, 1403)\...  0.7810
7   (1, 2)      1    (0, 1394)\t0.39145456656817385\n  (0, 1403)\...  0.7805
14  (1, 3)      1    (0, 3198)\t0.33933345622928995\n  (0, 3217)\...  0.7790
15  (1, 3)      2    (0, 3198)\t0.33933345622928995\n  (0, 3217)\...  0.7790
4   (1, 1)    100    (0, 159)\t0.6729033873983855\n  (0, 2021)\t0...  0.7790
17  (1, 3)     50    (0, 3198)\t0.33933345622928995\n  (0, 3217)\...  0.7785

___
## best model
Объединение признаков также не дало результатов. В итоге лучшая модель: лемматизация + Tfidf c 1-2 словограммами + LinearSVC

In [92]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
best_model = Pipeline([
        ('vectorizer', TfidfVectorizer(ngram_range=(1,2))),
        ('classifier', LinearSVC(random_state=random_state))
    ])
print(
    'cross-validation accuracy: %.3f\n' %
     cross_val_score(best_model, train_lemmatized.text, train_lemmatized.label, scoring='accuracy').mean()
)
X_train, X_test, y_train, y_test = train_test_split(train_lemmatized.text, train_lemmatized.label, random_state=random_state)
best_model.fit(X_train, y_train)
predicted = best_model.predict(X_test)
print(classification_report(y_test, predicted))

cross-validation accuracy: 0.792

              precision    recall  f1-score   support

           0       0.77      0.64      0.70       180
           1       0.81      0.89      0.85       320

    accuracy                           0.80       500
   macro avg       0.79      0.77      0.78       500
weighted avg       0.80      0.80      0.80       500



In [101]:
test_lemmatized = df_test.copy(deep=True)
test_lemmatized.text = test_lemmatized.text.apply(wordnet_lemmatize)
best_model.fit(train_lemmatized.text, train_lemmatized.label)
test_lemmatized['y'] = best_model.predict(test_lemmatized.text)

In [107]:
test_lemmatized[['Id', 'y']].to_csv('products_sentiment.csv', index=False)